In [20]:
import time
import sys
import qrcode
from projectq import MainEngine
from projectq.ops import H, Measure, All
from projectq.backends import Simulator
from projectq.cengines import TagRemover
from pyzbar.pyzbar import decode
from PIL import Image

# QR code generation and display
def generate_qr_code(data, title):
    qr = qrcode.make(data)
    qr.save(f"{title}.png")
    qr.show(title=title)
    return f"{title}.png"

# Read QR code and decode
def read_qr_code(filename):
    with Image.open(filename) as img:
        decoded_data = decode(img)[0].data.decode("utf-8")
    return decoded_data

# Function to calculate the error rate
def calculate_error_rate(original, final):
    errors = sum(1 for o, f in zip(original, final) if o != f)
    error_rate = (errors / len(original)) * 100
    return error_rate

# AES-like function with Hadamard gates applied and QR process
def AES_Hadamard(eng):
    # Step 1: Allocate quantum registers (8 qubits for each block and key)
    x0 = eng.allocate_qureg(8)  # 8 qubits for the plaintext block
    k = eng.allocate_qureg(8)   # 8 qubits for the key

    # Convert initial plaintext to QR code and read back
    plaintext_data = "11011001"  # Binary string for 8 qubits (example)
    plaintext_qr_filename = generate_qr_code(plaintext_data, "Plaintext QR Code")
    plaintext_from_qr = read_qr_code(plaintext_qr_filename)
    print("Text from Plaintext QR Code:", plaintext_from_qr)

    # Step 2: Apply Hadamard gates to all 8 qubits in x0 and k
    print('--- Applying Hadamard Gates to All 8 Qubits ---')
    sys.stdout.flush()
    All(H) | x0
    All(H) | k
    eng.flush()

    # Step 3: Measure the qubits after applying Hadamard gates
    print('--- Measuring the qubits after first Hadamard gates ---')
    All(Measure) | x0
    All(Measure) | k
    eng.flush()

    # Print and generate QR code for the initial key and plaintext after measurement
    print('Initial key (after Hadamard and measurement):')
    key_data = print_state(eng, k)
    key_qr_filename = generate_qr_code(key_data, "Key QR Code")
    key_from_qr = read_qr_code(key_qr_filename)
    print("Key from QR Code:", key_from_qr)

    print('Initial plaintext:')
    plaintext_data_measured = print_state(eng, x0)
    plaintext_measured_qr_filename = generate_qr_code(plaintext_data_measured, "Measured Plaintext QR Code")
    plaintext_measured_from_qr = read_qr_code(plaintext_measured_qr_filename)
    print("Measured Plaintext from QR Code:", plaintext_measured_from_qr)

    # Step 5: Apply Hadamard gates to all qubits again for another layer
    print('--- Applying Hadamard Gates Again (Second Layer) ---')
    sys.stdout.flush()
    All(H) | x0
    All(H) | k
    eng.flush()

    # Step 6: Final measurement and print final ciphertext
    print('--- Measuring Final Ciphertext ---')
    All(Measure) | x0
    All(Measure) | k
    eng.flush()

    print('Final Ciphertext:')
    final_ciphertext_data = print_state(eng, x0)

    # Convert final ciphertext to QR code and read back
    final_ciphertext_qr_filename = generate_qr_code(final_ciphertext_data, "Final Ciphertext QR Code")
    final_ciphertext_from_qr = read_qr_code(final_ciphertext_qr_filename)
    print("Ciphertext from Final QR Code:", final_ciphertext_from_qr)

    # Calculate and print the error rate
    error_rate = calculate_error_rate(plaintext_from_qr, final_ciphertext_from_qr)
    print(f"Error Rate: {error_rate:.2f}%")

    # Explicitly deallocate qubits
    del x0, k

# Helper function to print the quantum states in hex and binary format
def print_state(eng, qubits):
    eng.flush()
    state = [int(qubit) for qubit in qubits]  # Get the measured state (0 or 1)

    # Convert the state to a binary string
    binary_str = ''.join(str(bit) for bit in state)
    # Print as hex (2 characters per byte)
    hex_str = hex(int(binary_str, 2))[2:].zfill(len(qubits) // 4)
    print(f'Hex: {hex_str.upper()}')
    print(f'Binary: {binary_str}')
    sys.stdout.flush()

    return binary_str  # Return binary string for QR code generation

# Main function to run the AES-like function with Hadamard gates and QR process
def main():
    start_time = time.time()  # Start measuring time

    # Create a quantum engine with a full quantum simulator backend
    sim = Simulator()  # Use full quantum Simulator instead of ClassicalSimulator
    eng = MainEngine(backend=sim, engine_list=[TagRemover()])  # Simplified engine list

    # Run the AES-like quantum circuit
    AES_Hadamard(eng)

    # End measuring time
    end_time = time.time()
    execution_time = end_time - start_time
    print(f'\nExecution Time: {execution_time:.4f} seconds')
    sys.stdout.flush()


# Run the main function
if __name__ == '__main__':
    main()


(Note: This is the (slow) Python simulator.)
Text from Plaintext QR Code: 11011001
--- Applying Hadamard Gates to All 8 Qubits ---
--- Measuring the qubits after first Hadamard gates ---
Initial key (after Hadamard and measurement):
Hex: 1D
Binary: 00011101
Key from QR Code: 00011101
Initial plaintext:
Hex: AF
Binary: 10101111
Measured Plaintext from QR Code: 10101111
--- Applying Hadamard Gates Again (Second Layer) ---
--- Measuring Final Ciphertext ---
Final Ciphertext:
Hex: DB
Binary: 11011011
Ciphertext from Final QR Code: 11011011
Error Rate: 12.50%

Execution Time: 18.0823 seconds
